## Word-level Convolutional Neural Network for Sentence Classification

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset

print(torch.__version__)

1.0.1.post2


## Load data

이제 앞서 만들어둔 word vector 를 이용하여 Yoon Kim, 2015 의 word - level CNN sentence classifier 를 만들어봅니다.

In [2]:
import config
import numpy as np
from navermovie_comments import load_comments_image

class CommentsImage(Dataset):
    def __init__(self, large=False, max_len=20):
        super(CommentsImage, self).__init__()
        # use only this tokenizer
        tokenize = 'soynlp_unsup'

        self.X, labels, self.idx_to_vocab = load_comments_image(
            large=large, tokenize=tokenize, max_len=max_len)

        # {0: negative, 1: neutral, 2: positive}
        labels[np.where(labels <= 4)[0]] = 0
        labels[np.where((labels > 4) & (labels < 8))] = 1
        labels[np.where(8 <= labels)[0]] = 2
        self.labels = labels

        # transform numpy.ndarray to torch.tensor
        self.X = torch.LongTensor(self.X)
        self.labels = torch.LongTensor(self.labels)

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, index):
        X = self.X[index]
        y = self.labels[index]
        return X, y


large = True
data_name = 'large' if large else 'small'
comments_images = CommentsImage(large)

print(len(comments_images))

3279677


`93234` 은 padding idx 입니다.

In [3]:
X, y = comments_images[2]
print(X)
print(y)

tensor([17358,   149,    83,   324, 93234, 93234, 93234, 93234, 93234, 93234,
        93234, 93234, 93234, 93234, 93234, 93234, 93234, 93234, 93234, 93234])
tensor(2)


앞서 학습한 word2vec model 에서 word vectors 도 가져옵니다.

In [4]:
from navermovie_comments import load_trained_embedding

word2vec_model = load_trained_embedding(data_name=data_name,
    tokenize='soynlp_unsup', embedding='word2vec')

wv = word2vec_model.wv.vectors

print(wv.shape)
print(len(comments_images.idx_to_vocab))

(93234, 100)
93235


padding 을 위한 zero vectors 를 추가합니다.

In [5]:
zero_vector = np.zeros((1, wv.shape[1]), dtype=wv.dtype)
wv = np.vstack([wv, zero_vector])
wv.shape

(93235, 100)

사용할 parameter 몇 가지를 미리 정의합니다. 

Yoon Kim (2015) 의 논문에서는 각 크기의 kernel 마다 100 개의 CNN filter 를 만들었습니다. 이를 num_filters 에 저장해둡니다. 

In [6]:
# data related parameter
num_words, embedding_dim = wv.shape
num_filters = 100
num_classes = np.unique(comments_images.labels.numpy()).shape[0]
n_data = len(comments_images)

# training related parameter
epochs = 10
batch_size = 128
learning_rate = 0.002

Data Loader 를 만듭니다.

In [7]:
batch_size = 64
use_cuda = False
kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}

data_loader = torch.utils.data.DataLoader(
    comments_images, batch_size=batch_size, shuffle=True, **kwargs)

## Model

이번 예시는 아래의 reference 를 참고하였습니다. 유명한 모델들은 좋은 구현체들이 많습니다. 검색한 뒤, 본인의 스타일에 잘 맞는 코드를 이용하시면 좋습니다.

reference : https://github.com/castorini/Castor/tree/master/kim_cnn

Yoon Kim (2014) 에서는 (1) word embedding vectors 도 task 에 맞춰 학습하는 non-static version 과 (2) pretrained word embedding vectors 는 고정하는 static, (3) embedding vectors 를 random initialization 하거나 (4) static, non-static 을 합쳐쓰는 방법을 제안하였습니다. 두 종류의 word embedding vectors 를 함께 이용한다면 lookup 하는 과정에서 2 channel images 를 만들면 됩니다. 우리는 CNN 의 구조부터 익히기 위하여 static / non-static single channel 방식으로 모델을 만들어 봅니다.

### init 함수

init 에는 단어의 임베딩 벡터 차원의 크기, 단어 개수, 필터 개수, 클래스 개수를 각각 입력받습니다. 이들은 CNN 과 fully connected layer 의 input - output dimension 을 정의하는데 필요합니다. pretrained_wordvec 은 gensim 이나 다른 알고리즘으로 학습된 word embedding vectors 입니다. Dropout-ratio 는 dropout 하는 units 의 비율입니다. 

입력된 embedding_dim 과 num_words 가 pretrained word vector 의 크기와 같은지 확인하고, Embedding layer 를 만든 뒤, 이 값을 복사합니다.

```python
class TextCNN(nn.Module):
    def __init__(self, parameters ... ):
        # ...
        n, m = pretrained_wordvec.shape
        assert n == num_words and m == embedding_dim
        self.embed = nn.Embedding(num_words, embedding_dim)
        self.embed.weight.data.copy_(torch.from_numpy(pretrained_wordvec))
```

Yoon Kim, 2015 에서는 3, 4, 5 - gram 의 CNN filter 를 각각 100 개씩, 총 300 개의 filter 를 거친 뒤, dropout 과 fully connected layer 를 거쳐 softmax 로 classification 을 하였습니다. 우리는 한국어 데이터를 적용할 것이고, bigram 도 중요한 feature 이기 때문에 2, 3, 4 - gram 을 이용하는 CNN filter 를 만듭니다. Dropout layer 는 dropout_ratio 만 입력하면 됩니다. Fully connected layer 에 입력되는 값은 세 종류의 필터 각각 100 개씩, 총 300 개의 필터가 적용된 값입니다. (3 * num_filters, num_classes) 의 weight 를 지니는 fully connected layer 를 이용합니다.

```python
class TextCNN(nn.Module):
    def __init__(self, parameters ... ):
        # ...
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=num_filters, kernel_size=(2, embedding_dim))
        self.conv2 = nn.Conv2d(in_channels=1, out_channels=num_filters, kernel_size=(3, embedding_dim))
        self.conv3 = nn.Conv2d(in_channels=1, out_channels=num_filters, kernel_size=(4, embedding_dim))

        self.dropout = nn.Dropout(dropout_ratio)
        self.fc1 = nn.Linear(3 * num_filters, num_classes)
```

### forward 함수 구현

우리는 세 종류의 CNN filter 를 각각 self.conv1, self.conv2, self.conv3 으로 만들었습니다. forward 함수에 입력된 x 가 각각의 convolution layer 를 거치면 각각의 output 값이 만들어집니다. 이 값을 concatenation 하여 fully connected layer 에 입력하는 input 으로 만들 것입니다.

일단 입력된 단어열에 Embedding lookup 을 한 뒤, resize 를 합니다. unsqueese 를 하는 이유는 앞서 말한 바와 같이 torch.nn.Embedding 에 적용된 값은 channel 의 개수가 지정되지 않기 때문입니다. unsqueeze(1) 을 통하여 out 의 format 을 (bath, sent_len, embed_dim) 에서 (batch, channel_input, sent_len, embed_dim) 으로 변환합니다.

```python
class TextCNN(nn.Module):
    def forward(self, x):
        # ...
    out = self.embed(x) # (batch, sent_len, embed_dim)
    out = out.unsqueeze(1) # (batch, channel_input, sent_len, embed_dim)
```

Embedding lookup 을 통하여 만들어진 sentence image, out 을 세 종류의 CNN filter 에 적용합니다. 이 값을 list 에 모두 저장합니다. 각각의 CNN filter 가 적용된 값에 max pooling 을 적용합니다. 이 과정에서 squeeze, unsqueeze 를 왜 하는지 햇갈리시다면, 바로 위 chapter, applying CNN and pooling 을 다시 보시기 바랍니다. 현재 out 의 형태는 list of torch.Tensor 입니다. 이를 하나의 tensor 로 concatenation 합니다.

```python
class TextCNN(nn.Module):
    def forward(self, x):
        # ...
        out = [F.relu(self.conv1(out)).squeeze(3),
               F.relu(self.conv2(out)).squeeze(3),
               F.relu(self.conv3(out)).squeeze(3)]

        out = [F.max_pool1d(i, i.size(2)).squeeze(2) for i in out]
        out = torch.cat(out, 1)
```

이 값을 dropout 을 거치고, fully connected layer 에 넣습니다.

```python
class TextCNN(nn.Module):
    def forward(self, x):
        # ...
        out = self.dropout(out)
        logit = self.fc1(out) # (batch, target_size)

        return logit
```

In [8]:
class TextCNN(nn.Module):

    def __init__(self, embedding_dim, num_words, num_filters,
                 num_classes, pretrained_wordvec, dropout_ratio=0.5):

        super(TextCNN, self).__init__()

        self.embed = nn.Embedding(num_words, embedding_dim)

        # check word embedding vector shape
        n, m = pretrained_wordvec.shape
        assert n == num_words and m == embedding_dim

        self.embed.weight.data.copy_(torch.from_numpy(pretrained_wordvec))

        # in_channels, out_channels, 
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=num_filters, kernel_size=(2, embedding_dim), bias=False)
        self.conv2 = nn.Conv2d(in_channels=1, out_channels=num_filters, kernel_size=(3, embedding_dim), bias=False)
        self.conv3 = nn.Conv2d(in_channels=1, out_channels=num_filters, kernel_size=(4, embedding_dim), bias=False)

        self.dropout = nn.Dropout(dropout_ratio)
        # three type convolution. each conv. has num_filters
        self.fc1 = nn.Linear(3 * num_filters, num_classes)

    def forward(self, x):
        """x: sentence image
                torch.LongTensor"""

        out = self.embed(x) # (batch, sent_len, embed_dim)
        out = out.unsqueeze(1) # (batch, channel_input, sent_len, embed_dim)

        # three convolution filter
        out = [F.relu(self.conv1(out)).squeeze(3),
               F.relu(self.conv2(out)).squeeze(3),
               F.relu(self.conv3(out)).squeeze(3)]

        # 1 - max pooling for each conv
        out = [F.max_pool1d(i, i.size(2)).squeeze(2) for i in out]

        # concatenation
        out = torch.cat(out, 1)

        # dropout
        out = self.dropout(out)

        # fully connected neural network
        logit = self.fc1(out) # (batch, target_size)

        return logit

이제 모델을 만듭니다. 그리고 print(model) 을 하면 직접 만든 모델의 구조가 출력됩니다. 

In [9]:
model = TextCNN(
    embedding_dim,
    num_words,
    num_filters,
    num_classes,
    wv
)

print(model)

TextCNN(
  (embed): Embedding(93235, 100)
  (conv1): Conv2d(1, 100, kernel_size=(2, 100), stride=(1, 1), bias=False)
  (conv2): Conv2d(1, 100, kernel_size=(3, 100), stride=(1, 1), bias=False)
  (conv3): Conv2d(1, 100, kernel_size=(4, 100), stride=(1, 1), bias=False)
  (dropout): Dropout(p=0.5)
  (fc1): Linear(in_features=300, out_features=3, bias=True)
)


학습 함수는 이전의 document classification 예시와 비슷합니다. Data Loader 를 이용하였기 때문에 minibatch 단위로 학습이 됩니다. 매 epoch 마다 함수를 호출하는 형식으로 구현하였습니다.

In [56]:
def train(model, loss_func, optimizer, data_loader, epoch):

    n_data = len(data_loader)
    loss_sum = 0

    for i, (X, y) in enumerate(data_loader):

        # clean gradient buffer zero
        optimizer.zero_grad()

        # predict
        y_pred = model(X)

        # compute loss & back-propagation
        loss = loss_func(y_pred, y)
        loss.backward()
        optimizer.step()

        # cumulate loss
        loss_sum += loss.data.numpy()

        if i % 10 == 0:
            loss_tmp = loss_sum / (i+1)
            print('\repoch = {}, batch = {}, training loss = {}'.format(
                epoch, i, '%.3f' % loss_tmp), end='', flush=True)

    print('\repoch = {}, training loss = {}{}'.format(
        epoch, '%.3f' % (loss_sum / (i+1)), ' '*40), flush=True)

    return model

이제 model 과 train 함수를 모두 만들었으니 실제로 모델을 학습합니다.

## static vs non-static

Yoon Kim, 2015 에서의 static model 은 pretrained word vector 를 Embedding 의 값으로 이용하고, 더는 학습을 하지 않는 것입니다. Word embedding layer 는 network 의 맨 앞의 layer 입니다. 이 layer 에 gradient 가 전달되지 않도록 지정하면 word vectors 는 바뀌지 않습니다. 이전의 PyTorch 0.3x 까지는 Variable 을 만들 때 gradient 설정을 하기도 하였습니다. 지금은 requires_grad 를 True, False 로 지정해줘도 됩니다.

```python
model = TextCNN(embedding_dim, num_words, num_filters, num_classes, wordvec)
model.embed.weight.requires_grad = False
```

In [57]:
# Loss and optimizer
# model = TextCNN(embedding_dim, num_words, num_filters, num_classes, wv)

loss_func = nn.CrossEntropyLoss()  
optimizer = torch.optim.SGD(
    model.parameters(),
    lr=0.02
)

# do not update word embedding vector if static mode
model.embed.weight.requires_grad = False

for epoch in range(1, 50 + 1):
    model = train(model, loss_func, optimizer, data_loader, epoch)
    if epoch % 5 == 0:
        print()

epoch = 1, training loss = 0.451                                        
epoch = 2, training loss = 0.431                                        
epoch = 3, training loss = 0.424                                        
epoch = 4, training loss = 0.421                                        
epoch = 5, training loss = 0.418                                        

epoch = 6, training loss = 0.416                                        
epoch = 7, training loss = 0.414                                        
epoch = 8, training loss = 0.412                                        
epoch = 9, training loss = 0.411                                        
epoch = 10, training loss = 0.410                                        

epoch = 11, training loss = 0.409                                        
epoch = 12, training loss = 0.408                                        
epoch = 13, training loss = 0.408                                        
epoch = 14, training loss = 0.407            

In [59]:
for epoch in range(1, 100 + 1):
    model = train(model, loss_func, optimizer, data_loader, epoch)
    if epoch % 5 == 0:
        print()

epoch = 1, training loss = 0.396                                        
epoch = 2, training loss = 0.396                                        
epoch = 3, training loss = 0.395                                        
epoch = 4, training loss = 0.395                                        
epoch = 5, training loss = 0.395                                        

epoch = 6, training loss = 0.395                                        
epoch = 7, training loss = 0.395                                        
epoch = 8, training loss = 0.395                                        
epoch = 9, training loss = 0.395                                        
epoch = 10, training loss = 0.395                                        

epoch = 11, training loss = 0.394                                        
epoch = 12, training loss = 0.394                                        
epoch = 13, training loss = 0.394                                        
epoch = 14, training loss = 0.394            

KeyboardInterrupt: 

## performance

학습 성능을 확인합니다. 모델링에 집중하기 위하여 학습 / 테스트 데이터를 나누는 것은 하지 않았습니다. 이번에 우리가 측정하는 것은 training accuracy 입니다. data loader 를 입력하고 데이터의 앞 부분의 batch 에 대해서만 정확도를 측정하였습니다.

with torch.no_grad() 를 이용하면 이 때에는 gradient 계산을 하지 않습니다. model.eval 과 비슷한 역할을 합니다.

```python
with torch.no_grad():
    # ...
```

model 을 이용하여 `y_pred` 를 계산하고, torch.max 를 이용하여 각 row 마다 column 기준으로 max 값을 찾습니다. score, predicated 는 (max, argmax) 와 같습니다. predicated 와 y 가 같은 개수를 계산하여 n_correct 에 누적합니다.

```python
for X, y in data_loader:
    y_pred = model(X)
    score, predicted = torch.max(y_pred.data, dim=1)
    n_correct += (predicted == y).sum().numpy()
```

n_correct 의 개수를 batch number 와 batch size 의 곲으로 나눠주면 학습 정확도를 계산할 수 있습니다.

```python
accuracy = n_correct / (i * batch_size)
```

In [60]:
def accuracy_test(model, data_loader, first_batch=1000):
    with torch.no_grad():
        n_correct = 0
        for i, (X, y) in enumerate(data_loader):
            if i == first_batch:
                break
            y_pred = model(X)
            score, predicted = torch.max(y_pred.data, dim=1)
            n_correct += (predicted == y).sum().numpy()
            print('\r%d batch ...' % i, end='')
        print('\r%d batch was done' % i)

    accuracy = n_correct / (i * batch_size)
    print('accuracy = {}'.format(accuracy))

accuracy_test(model, data_loader)

1000 batch was done
accuracy = 0.84815625


모델의 parameter 값은 torch.save 를 이용하여 저장할 수 있습니다.

In [61]:
# path = './textcnn_params.pt'
# torch.save(model.state_dict(), path)